In [112]:
#import mygrad as mg
#loss = mg.mean(mg.maximum(0, margin - y * (x1 - x2)))
import numpy as np
from gensim.models import KeyedVectors
from cogworks_data.language import get_data_path
from pathlib import Path
import json
import re, string
from collections import Counter
import numpy as np
import math
import pickle

In [1]:
# load COCO metadata
filename = get_data_path("captions_train2014.json")
with Path(filename).open() as f:
    coco_data = json.load(f)

In [9]:
filename = "glove.6B.200d.txt.w2v"

# this takes a while to load -- keep this in mind when designing your capstone project
glove = KeyedVectors.load_word2vec_format(get_data_path(filename), binary=False)

In [83]:
punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
def strip_punc(corpus):
    return punc_regex.sub('', corpus)

text = "Apples rule. Apples are the best. Truly, they are. Truly... Truly"
text = strip_punc(text).lower()
tokenized = text.split()
tokens = Counter(tokenized)
#descriptor = np.array([tokens[word] for word in sorted(tokens)], dtype=float)
#descriptor
#tokens

**Embed Caption**

In [16]:
def to_counter(doc):
    return Counter(strip_punc(doc).lower().split())

In [40]:
def to_vocab(counters, k=None):
    output = set()
    for i in counters:
        output.add(i)
    #output = set(output)
    return sorted(output)

In [18]:
def to_idf(vocab, counters):
    N = len(counters)
    nt = [sum(1 if term in counter else 0 for counter in counters) for term in vocab]
    nt = np.array(nt, dtype=float)

    return np.log10(N/nt)    

In [78]:
def embed_caption(caption):
    caption = to_counter(caption)
    caption_idf = to_idf(to_vocab(caption), caption)
    caption_glove = [glove[i] for i in caption]

    dots=[]
    for i in range(len(caption_idf)):
        dots.append(np.dot(caption_idf[i], caption_glove[i]))        
    summation = sum(dots)
    unit = summation / math.sqrt(sum(summation**2))

    return unit

**Image Descriptor**

In [85]:
with Path(get_data_path('resnet18_features.pkl')).open('rb') as f:
    resnet18_features = pickle.load(f)

In [96]:
resnet18_features[57870][0].shape

(512,)

**Creating Training Data**

In [116]:
training_data = []
index=0
#(true-caption-ID, true-image-ID, confusor-image-ID)
for i in coco_data["annotations"]:
    while True:
        index = np.random.randint(len(coco_data["annotations"]))
        if (coco_data["annotations"][index]["image_id"]!=i["image_id"]):
            break
    training_data.append((i["id"], i["image_id"], coco_data["annotations"][index]["image_id"]))

In [118]:
len(training_data)

414113